In [1]:
import operator
from collections import defaultdict
from time import time

import numpy as np

from sklearn.cluster import MiniBatchKMeans, SpectralCoclustering
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import v_measure_score

Un biclúster es un subconjunto de filas y columnas de una matriz de datos que muestra una coherencia o similitud significativa entre sí y es útil para descubrir patrones subyacentes en conjuntos de datos multidimensionales

Spectral Co-clustering: busca identificar grupos coherentes tanto en las filas como en las columnas de la matriz.
La idea principal es encontrar submatrices dentro de la matriz de datos original donde las filas y las columnas están altamente
correlacionadas entre sí, haya patrones intrínsecos en los datos que pueden no ser evidentes mediante otros métodos de agrupamiento<p>
Spectral: se refiere al uso de técnicas espectrales en el proceso de agrupamiento. Las técnicas espectrales involucran el análisis de las propiedades de los eigenvalores y eigenvectores de una matriz, en este caso, la matriz de datos.

La tokenización es el proceso de dividir una cadena de texto en unidades más pequeñas llamadas tokens. 
Estos tokens pueden ser palabras individuales, caracteres, o incluso frases completas,
dependiendo del nivel de granularidad deseado.
- Creamos el tokenizador y las categorias:

In [2]:
def number_normalizer(tokens): #Cuando exista algún token numérico, la función revisa si el primer caracter es un número y si es así lo sustituye por #NUMBER(marcador de posición)
    """Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

class NumberNormalizingVectorizer(TfidfVectorizer): #Crea un tokenizador, convierte un doc de texto a una matriz TF-IDF
    def build_tokenizer(self): #modifica el metodo super para personalizar el tokenizador
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc))) #devuelve el resultado de una lambda, en la que coge un doc, lo tokeniza y normaliza con el metodoanterior
categories = [ #define las categorias, que parece ser un grupo de noticias
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
]

La descomposición de valores singulares es una técnica matemática fundamental que se utiliza en muchos algoritmos de aprendizaje automático,
incluidos algunos métodos de agrupamiento y reducción de dimensionalidad.<p>
Definimos dos algoritmos diferentes de agrupación:

In [3]:
newsgroups = fetch_20newsgroups(categories=categories) #cargamos los grupos de noticias con el metodo
y_true = newsgroups.target #se asigna la matriz de las etiquetas que contiene el doc

vectorizer = NumberNormalizingVectorizer(stop_words="english", min_df=5) #instanciamos la clase creada anteriormente, eliminamos las palabras en ingles y vectoriza el doc
cocluster = SpectralCoclustering( #Realizamos el co-clustering espectral
    n_clusters=len(categories), svd_method="arpack", random_state=0 #Tantos clusteres como categorias tengamos / 
                                                                    #svd: método que se utilizará para la descomposición de valores singulares
)
kmeans = MiniBatchKMeans( #kmeans optimizado para grandes volumenes de datos,en lugar de utilizar todo el conjunto de datos en cada iteración, utiliza pequeños lotes de datos seleccionados aleatoriamente
    n_clusters=len(categories), batch_size=20000, random_state=0, n_init=3
)

Vectorizamos y ajustamos los datos para utilizarlos en ambos algoritmos y evaluarlos:

In [4]:
print("Vectorizing...")
X = vectorizer.fit_transform(newsgroups.data)

print("Coclustering...")
start_time = time()
cocluster.fit(X)
y_cocluster = cocluster.row_labels_ 
print(
    "Done in {:.2f}s. V-measure: {:.4f}".format(
        time() - start_time, v_measure_score(y_cocluster, y_true)
    )
)

print("MiniBatchKMeans...")
start_time = time()
y_kmeans = kmeans.fit_predict(X)
print(
    "Done in {:.2f}s. V-measure: {:.4f}".format(
        time() - start_time, v_measure_score(y_kmeans, y_true)
    )
)

feature_names = vectorizer.get_feature_names_out()  #obtenemos los features
document_names = list(newsgroups.target_names[i] for i in newsgroups.target) #obtenemos la lista de las categorias

Vectorizing...
Coclustering...
Done in 0.54s. V-measure: 0.4415
MiniBatchKMeans...
Done in 0.98s. V-measure: 0.3015


El corte normalizado es una métrica utilizada para evaluar la calidad de un biclúster en un problema de co-clustering.

In [5]:
def bicluster_ncut(i): #calcula el corte normalizado para un biclúster específico identificado por el índice i
    rows, cols = cocluster.get_indices(i)
    
    if not (np.any(rows) and np.any(cols)): #comprueba si hay elementos en las filas y columnas del biclúster. Si no hay elementos en ninguna de las filas o columnas,
                                            #devuelve un valor máximo de punto flotante, indicando que el biclúster no es válido  
        import sys

        return sys.float_info.max

    #obtienen los índices complementarios al biclúster i:
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    
    # Note: the following is identical to X[rows[:, np.newaxis],
    # cols].sum() but much faster in scipy <= 0.16
    weight = X[rows][:, cols].sum() #Calcula el peso del biclúster, que es la suma de los valores en las filas y columnas del biclúster
    cut = X[row_complement][:, cols].sum() + X[rows][:, col_complement].sum() #Calcula el corte del biclúster: suma de los valores en las filas y columnas complementarias
    
    return cut / weight #Devuelve el corte normalizado


def most_common(d): #ordena de forma descendente según el segundo valor del diccionario
    """Items of a defaultdict(int) with the highest values.

    Like Counter.most_common in Python >=2.7.
    """
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)


bicluster_ncuts = list(bicluster_ncut(i) for i in range(len(newsgroups.target_names))) #Crea una lista que contiene los cortes normalizados para cada biclúster
best_idx = np.argsort(bicluster_ncuts)[:5] #ordena los ordenar los índices de los biclústeres según sus cortes normalizados en orden ascendente y luego selecciona los primeros cinco índices

Imprimimos la información sobre los mejores biclústeres identificados en el conjunto de datos de los 20 grupos de noticias:

In [6]:
print()
print("Best biclusters:")
print("----------------")
for idx, cluster in enumerate(best_idx):
    n_rows, n_cols = cocluster.get_shape(cluster) #Obtiene el número de filas y columnas del biclúster
    cluster_docs, cluster_words = cocluster.get_indices(cluster) #Obtiene los índices de los documentos y palabras que pertenecen al biclúster
    if not len(cluster_docs) or not len(cluster_words):
        continue

    # categories
    counter = defaultdict(int) #crea un diccionario donde el valor predeterminado de cada elemento es 0, para contar la frecuencia de las categorías de documentos dentro del biclúster
    for i in cluster_docs:
        counter[document_names[i]] += 1
    cat_string = ", ".join(
        "{:.0f}% {}".format(float(c) / n_rows * 100, name) #calcula el porcentaje de documentos que hay en cada categoría dentro del biclúster
        for name, c in most_common(counter)[:3]
    )

    # words
    out_of_cluster_docs = cocluster.row_labels_ != cluster #indica qué documentos no pertenecen al biclúster actual
    out_of_cluster_docs = np.where(out_of_cluster_docs)[0] #obtenemos los índices de los documentos que no pertenecen al biclúster
    word_col = X[:, cluster_words] #Selecciona las columnas relevantes del vectorizado, corresponden a las palabras presentes en el biclúster actual
    word_scores = np.array( #calculan las puntuaciones de importancia de las palabras dentro del biclúster
        word_col[cluster_docs, :].sum(axis=0)
        - word_col[out_of_cluster_docs, :].sum(axis=0)
    )
    word_scores = word_scores.ravel()
    important_words = list(
        feature_names[cluster_words[i]] for i in word_scores.argsort()[:-11:-1] #Selecciona las 10 palabras más importantes dentro del biclúster
    )

    print("bicluster {} : {} documents, {} words".format(idx, n_rows, n_cols))
    print("categories   : {}".format(cat_string))
    print("words        : {}\n".format(", ".join(important_words)))


Best biclusters:
----------------
bicluster 0 : 8 documents, 6 words
categories   : 100% talk.politics.mideast
words        : cosmo, angmar, alfalfa, alphalpha, proline, benson

bicluster 1 : 1948 documents, 4325 words
categories   : 23% talk.politics.guns, 18% talk.politics.misc, 17% sci.med
words        : gun, guns, geb, banks, gordon, clinton, pitt, cdt, surrender, veal

bicluster 2 : 1259 documents, 3534 words
categories   : 27% soc.religion.christian, 25% talk.politics.mideast, 25% alt.atheism
words        : god, jesus, christians, kent, sin, objective, belief, christ, faith, moral

bicluster 3 : 775 documents, 1623 words
categories   : 30% comp.windows.x, 25% comp.sys.ibm.pc.hardware, 20% comp.graphics
words        : scsi, nada, ide, vga, esdi, isa, kth, s3, vlb, bmug

bicluster 4 : 2180 documents, 2802 words
categories   : 18% comp.sys.mac.hardware, 16% sci.electronics, 16% comp.sys.ibm.pc.hardware
words        : voltage, shipping, circuit, receiver, processing, scope, mpce, an